In [ ]:
def receive(pow_noise = 0, vct_h = np.array([1]), show_fig = True):
  
  vct_bits_source = source(SYMBOLS)
  vct_coded_bits = code(vct_bits_source)
  vct_ak = modulate(vct_coded_bits)
  vct_ak_for_conv = upsample(transform(vct_ak, True), UPS_VALUE, True)
  vct_h_for_conv = downsample(vct_h, SAMPLESxSYM//UPS_VALUE)
  [vct_processed, vct_processed_x_axis, vct_convolved] = process_chan(pow_noise, vct_ak_for_conv, vct_h_for_conv)
  [vct_ADCout, vct_ADCout_x_axis] = convert_ADC(vct_processed)
  [vct_iter, vct_mu, vct_interp, vct_mk_plus_mu, vct_error_n, vct_error_n_x, vct_W] = sync_recover(vct_ADCout, vct_ADCout_x_axis) # Synchronism recover
  vct_out_sync = sync_detect(vct_interp) # detection of symbols path
  [vct_detected_symbs, vct_c, Ek_SAMPLES, Q_SAMPLES, vct_MSE_symb, vct_MSE_coef, matrix_c, vct_c_opt] = equalize(vct_out_sync, vct_h, pow_noise, vct_ak)
  vct_demodulated_symbs = demodulate(vct_detected_symbs)
  vct_decoded_bits = decode(vct_demodulated_symbs)

  ber_point = 0; ser_point = 0; # this line is only for debug or cases in which function below is not implemented
  [ber_point, ser_point] = estimate_errors(vct_bits_source, vct_decoded_bits, vct_ak, vct_detected_symbs, pow_noise) # BER and SER

  # Figures
  if show_fig == True:

    eye_diagram(vct_processed, eyes = 2) # Eye diagrams of real and imaginary parts

    if SYMBOLS*UPS_VALUE > 20000:
      print('Cuidado. Usa demasiados recursos gráficos')
      raise ValueError

    # Stem plot of the equalizer coefficients
    plt.figure(9, ([20, 5])); 
    plt.subplot(1,2,1)
    plt.title('Equalizer coefficients. Real part.')
    plt.stem(vct_c.real, use_line_collection=True)
    plt.grid(True)
    plt.subplot(1,2,2)
    plt.title('Equalizer coefficients. Imaginary part.')
    plt.stem(vct_c.imag, use_line_collection=True)
    plt.grid(True)

    # MSE figures. 
    plt.figure(num=1, figsize=(18, 5), dpi= 60, facecolor='w', edgecolor='k')

    plt.subplot(1,2,1)
    plt.semilogy(vct_MSE_symb, 'r-.')
    plt.xlabel('Transmitted symbols')
    plt.xlim([0, SYMBOLS-Ek_SAMPLES -N])
    plt.title('Mean Square Error (MSE) among transmitted and detected symbols')
    plt.grid(True)

    plt.subplot(1,2,2)     
    plt.semilogy(vct_MSE_coef, 'b-.')
    plt.xlabel('Transmitted symbols')
    plt.xlim([0, SYMBOLS-Q_SAMPLES -N])
    plt.title('Mean Square Error (MSE) among actual and optimum coefficients')
    plt.grid(True)

    # The variation of every Montecarlo coefficient and the optimal coefficients are plotted.
    plt.figure(num=3, figsize=(16, 8), dpi= 70, facecolor='w', edgecolor='k')
    ax1 = plt.subplot(1,2,1) # real parts of the coefficients
    ax1.set_title('Real parts of the coefficients')
    ax2 = plt.subplot(1,2,2) # imaginary parts of the coefficients
    ax2.set_title('Imaginary parts of the coefficients')

    for itf_2 in range(0, N):
      for itf_1 in range(0, SYMBOLS, SYMBOLS//250): # third arg to accelerate program execution
        
        ax1.grid(True)
        ax1.plot(itf_1, matrix_c[itf_2, itf_1].real, 'c.-')
        ax1.plot(itf_1, vct_c_opt[itf_2].real, 'g.-') # Vector asymptotes of the optimal coefficients.
        
        ax2.grid(True)
        ax2.plot(itf_1, matrix_c[itf_2, itf_1].imag, 'c.-')
        ax2.plot(itf_1, vct_c_opt[itf_2].imag, 'g.-') # Vector asymptotes of the optimal coefficients.

    # Plotly Figures
    figs = np.arange(1, 17) 
    ### Real part ###
    fig = make_subplots(rows=2, cols=1)
    if 1 in figs:
      fig.append_trace(go.Scatter(
          y= vct_processed.real,
          x= vct_processed_x_axis,
          marker_color='red', marker_symbol='circle', mode ='markers', name='ADC input. Real part.',
      ), row=1, col=1)
    if 2 in figs:
      fig.append_trace(go.Scatter(
          y= vct_ADCout.real,
          x= vct_ADCout_x_axis,
          marker_color='green', marker_symbol='square', mode ='markers', name='ADC output. Real part.',
      ), row=1, col=1)
    if 3 in figs:
      fig.append_trace(go.Scatter(
          y= vct_interp.real, 
          x= vct_mk_plus_mu,
          marker_color='yellow', marker_symbol='triangle-up', mode ='markers', name='Interpolator output. Real part.',
      ), row=1, col=1)
    # if 4 in figs:
    #   fig.append_trace(go.Scatter(
    #       y= vct_equalized.real, 
    #       x= vct_mk_plus_mu[0:-N+1:2],
    #       marker_color='coral', marker_symbol='triangle-down', mode ='markers', name='Equalized output. Real part.',
    #   ), row=1, col=1)
    if 5 in figs:
      fig.append_trace(go.Scatter(
          y= vct_out_sync.real, 
          x= vct_mk_plus_mu[UPS_VALUE//2-2::UPS_VALUE//2],
          marker_color='purple', marker_symbol='pentagon', mode ='markers', name='Symbols recovered. Real part.',
      ), row=1, col=1)
    if 6 in figs:
      fig.append_trace(go.Scatter(
          y= vct_error_n, 
          x= vct_error_n_x, 
          marker_color='cyan', name='TED error.',
      ), row=1, col=1)
    if 7 in figs:
      fig.append_trace(go.Scatter(
          y= vct_W, 
          x= vct_error_n_x, 
          marker_color='orange', name='W error.',
      ), row=1, col=1)
    if 8 in figs:
      fig.append_trace(go.Scatter(
          y= np.mod(vct_mu + 0.5, 1), # mu mapping
          x= np.arange(2, SYMBOLS*UPS_VALUE -2)*(1-FREQ_SHIFT_FACTOR),
          marker_color='olive', name='mu values.',
      ), row=1, col=1)
    if 9 in figs:
      fig.append_trace(go.Scatter(
          y= vct_ak_for_conv.real,
          x= np.arange(0, len(vct_ak_for_conv.real), 1),
          marker_color='blue', marker_symbol='cross', mode ='markers', name='Transmitted symbols. Real part.',
      ), row=1, col=1)
    if 10 in figs:
      fig.append_trace(go.Scatter(
          y= np.ones(len(vct_iter))*1.2,
          x= vct_iter,
          marker_color='black', marker_symbol='hexagon', mode = 'markers', name='Iter vector.',
      ), row=1, col=1)
    ### Imaginary part ###
    if 11 in figs:
      fig.append_trace(go.Scatter(
          y= vct_processed.imag,
          x= vct_processed_x_axis,
          marker_color='red', marker_symbol='circle', mode ='markers', name='ADC input. Imaginary part.',
      ), row=2, col=1)
    if 12 in figs:
      fig.append_trace(go.Scatter(
          y= vct_ADCout.imag, 
          x= vct_ADCout_x_axis,
          marker_color='green', marker_symbol='square', mode ='markers', name='ADC output. Imaginary part.',
      ), row=2, col=1)
    if 13 in figs:
      fig.append_trace(go.Scatter(
          y= vct_interp.imag, 
          x= vct_mk_plus_mu,
          marker_color='yellow', marker_symbol='triangle-up', mode ='markers', name='Interpolator output. Imaginary part.',
      ), row=2, col=1)
    # if 14 in figs:
    #   fig.append_trace(go.Scatter(
    #       y= vct_equalized.imag, 
    #       x= vct_mk_plus_mu[0:-N+1:2],
    #       marker_color='coral', marker_symbol='triangle-down', mode ='markers', name='Equalized output. Imaginary part.',
    #   ), row=2, col=1)
    if 15 in figs:
      fig.append_trace(go.Scatter(
          y= vct_out_sync.imag, 
          x= vct_mk_plus_mu[UPS_VALUE//2-2::UPS_VALUE//2],
          marker_color='purple', marker_symbol='pentagon', mode ='markers', name='Symbols recovered. Imaginary part.',
      ), row=2, col=1)
    if 16 in figs:
      fig.append_trace(go.Scatter(
          y= vct_ak_for_conv.imag,
          x= np.arange(0, len(vct_ak_for_conv.imag), 1),
          marker_color='blue', marker_symbol='cross', mode = 'markers', name='Transmitted symbols. Imaginary part.',
      ), row=2, col=1)
    fig.update_layout(height=600, width=1000, title_text='Signals')
    fig.update_xaxes(tickangle=0,
                    tickmode = 'array',
                    tickvals = np.arange(0, SYMBOLS*UPS_VALUE, UPS_VALUE),
                    ticktext= np.arange(1, SYMBOLS+1, 1).astype(int),)
    fig.show()

  return [ber_point, ser_point]